# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [22]:
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum
import datetime
import numpy as np
import pandas as pd


# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) write code to answer the quiz questions 

In [23]:
# instantiate a Spark session 
spark = SparkSession.builder.appName("Wrangling for quiz").getOrCreate()

In [24]:
spark

In [4]:
# read in the data set located at the path "data/sparkify_log_small.json"
path = "data/sparkify_log_small.json"
data = spark.read.json(path)
data.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



# Question 1

Which page did user id "" (empty string) NOT visit?

In [5]:
# TODO: write your code to answer question 1

In [6]:
not_visitted = set(data.select('page').collect()) - set(data.select(['page']).where(data.userId == "").collect()) 
for x in not_visitted:
    print(x.page)

Logout
Downgrade
Error
Submit Downgrade
Upgrade
Settings
NextSong
Save Settings
Submit Upgrade


# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [10]:
# TODO: use this space to explore the behavior of the user with an empty string
data.select("auth").where(data.userId == "").dropDuplicates().collect()

[Row(auth='Logged Out'), Row(auth='Guest')]

# Question 3

How many female users do we have in the data set?

In [44]:
# TODO: write your code to answer question 3
female_users = data.where(data.gender=="F").count()
female_users

3820

# Question 4

How many songs were played from the most played artist?

In [15]:
# TODO: write your code to answer question 4
data.filter(data.page == 'NextSong').select("Artist").groupBy("Artist").agg({"Artist":"count"}) \
.withColumnRenamed("count(Artist)", "Artistcount").sort(desc("Artistcount")).show(1)

+--------+-----------+
|  Artist|Artistcount|
+--------+-----------+
|Coldplay|         83|
+--------+-----------+
only showing top 1 row



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [21]:
# TODO: write your code to answer question 5

flag_homepage_visit = udf(lambda x: 1 if x == "Home" else 0, IntegerType())

windowval = Window.partitionBy("userId") \
            .orderBy(desc("ts")) \
            .rangeBetween(Window.unboundedPreceding, 0)

user_log_valid = data.filter((data.page == 'NextSong') | (data.page == 'Home')) \
    .select('userId', 'page', 'ts') \
    .withColumn("visited_home", flag_homepage_visit(col("page"))) \
    .withColumn("phase", Fsum("visited_home").over(windowval))


user_log_valid.where(data.page == 'NextSong') \
    .groupBy('userId', 'phase') \
    .agg({'phase':'count'}) \
    .agg({'count(phase)':'avg'}).show()

+-----------------+
|avg(count(phase))|
+-----------------+
|6.898347107438017|
+-----------------+

